In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Lambda, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import backend as K


In [2]:
max_seq_length = 100  # Length of input sequences
vocab_size = 10000    # Size of vocabulary
embedding_dim = 64    # Embedding dimensions
latent_dim = 32       # Latent space dimensions


In [3]:
# Encoder
input_text = Input(shape=(max_seq_length,))
x = Embedding(vocab_size, embedding_dim)(input_text)
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_sequences=False))
encoder_outputs = encoder_bilstm(x)

# VAE Sampling layer
def sampling(args):
    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

z_mean = Dense(latent_dim)(encoder_outputs)
z_log_sigma = Dense(latent_dim)(encoder_outputs)
z = Lambda(sampling)([z_mean, z_log_sigma])

# Instantiate encoder model
encoder = Model(input_text, [z_mean, z_log_sigma, z])
encoder.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 64)              640000    ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  (None, 64)                   24832     ['embedding[0][0]']           
 al)                                                                                              
                                                                                                  
 dense (Dense)               (None, 32)                   2080      ['bidirectional[0][0]']   

In [4]:
# Example text
texts = ["Hello world", "Variational Autoencoders are fun"]

# Tokenize text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
data = pad_sequences(sequences, maxlen=max_seq_length)


In [5]:
encoder.predict(data)

1/1 [==============================] - 1s 1s/step


[array([[ 0.01100398,  0.06712   , -0.01439812,  0.00046785, -0.05531935,
         -0.02647513,  0.04038674, -0.05212804, -0.03399178,  0.00518873,
         -0.01787976,  0.00167908, -0.00670539, -0.03547619, -0.01228551,
         -0.01111274, -0.00182266,  0.01015564, -0.02094446,  0.03367537,
         -0.00631543, -0.02280481,  0.05500431, -0.03394123, -0.01402383,
         -0.00435347, -0.03191426, -0.02499626,  0.01186818,  0.0457192 ,
          0.01166202,  0.0112183 ],
        [ 0.01658951,  0.06641938, -0.01758764, -0.00832391, -0.05476805,
         -0.00712335,  0.01531118, -0.04461461, -0.03675004, -0.00276413,
         -0.01810635, -0.00223366,  0.00014964, -0.03841908, -0.01330219,
         -0.0027553 , -0.00553959, -0.00743321, -0.00856722,  0.03303369,
          0.00187606, -0.02842741,  0.05847037, -0.00820848, -0.01121676,
         -0.01320982, -0.03578988, -0.02262063,  0.0084121 ,  0.04496844,
          0.00961072,  0.01755421]], dtype=float32),
 array([[ 0.00829639,  